In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field,BucketIterator
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter


c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] The specified procedure could not be found